This notebook shows how to convert the TF variant of a SegFormer model to ONNX for optimized inference on CPUs. 

## Installs

In [1]:
!pip install -Uqq tf2onnx
!pip install -Uqq onnxruntime
!pip install git+https://github.com/huggingface/transformers -q

     |████████████████████████████████| 440 kB 22.7 MB/s 
     |████████████████████████████████| 13.1 MB 23.6 MB/s 
     |████████████████████████████████| 4.9 MB 29.1 MB/s 
     |████████████████████████████████| 46 kB 4.1 MB/s 
     |████████████████████████████████| 86 kB 7.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 31.3 MB/s 
     |████████████████████████████████| 101 kB 7.9 MB/s 
     |████████████████████████████████| 596 kB 61.4 MB/s 


## Imports

In [2]:
from transformers import SegformerFeatureExtractor, TFSegformerForSemanticSegmentation

from PIL import Image
import numpy as np
import timeit

import onnx
import tf2onnx
import tensorflow as tf
import onnxruntime as ort

## Load model

You can know more about the SegFormer model from [here](https://huggingface.co/docs/transformers/main/en/model_doc/segformer). You can find all the pre-trained checkpoints of TensorFlow [here](https://huggingface.co/models?library=tf&other=segformer&sort=downloads).

In [3]:
model_ckpt = "nvidia/segformer-b5-finetuned-ade-640-640"

feature_extractor = SegformerFeatureExtractor.from_pretrained(model_ckpt)
model = TFSegformerForSemanticSegmentation.from_pretrained(model_ckpt)

All model checkpoint layers were used when initializing TFSegformerForSemanticSegmentation.

All the layers of TFSegformerForSemanticSegmentation were initialized from the model checkpoint at nvidia/segformer-b5-finetuned-ade-640-640.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFSegformerForSemanticSegmentation for predictions without further training.


## ONNX conversion

In [4]:
input_size = feature_extractor.size
input_signature = [
    tf.TensorSpec([None, 3, input_size, input_size], tf.float32, name="pixel_values")
]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=15)
onnx_model_path = model_ckpt.split("/")[-1] + ".onnx"
onnx.save(onnx_model, onnx_model_path)

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [5]:
!ls -lh {onnx_model_path}

-rw-r--r-- 1 root root 326M Jul 25 01:59 segformer-b5-finetuned-ade-640-640.onnx


## Compare ONNX predictions

In [6]:
dummy_inputs = tf.random.normal((1, 3, input_size, input_size))
dummy_inputs_numpy = dummy_inputs.numpy()

In [7]:
tf_outputs = model(dummy_inputs, training=False)

In [8]:
sess = ort.InferenceSession(onnx_model_path)
ort_outputs = sess.run(None, {"pixel_values": dummy_inputs_numpy})

In [9]:
list(tf_outputs.logits.shape) == list(ort_outputs[0].shape)

True

In [10]:
np.allclose(tf_outputs.logits.numpy(), ort_outputs, rtol=1e-5, atol=1e-05)

True

## Benchmarking speed

To benchmark the speed of these two models (TensorFlow and ONNX) we take the latency i.e., the time it takes to run a forward pass through the model. Here, we will be making such forward passes on a single data point for 25 times. We'll report the average latency. 

### TF model

In [11]:
# Warmup
print("Benchmarking TF model...")
for _ in range(2):
    _ = model(dummy_inputs, training=False)

# Timing
tf_runtimes = timeit.repeat(
    lambda: model(dummy_inputs, training=False), number=1, repeat=25
)
print(f"Average latency (seconds): {np.mean(tf_runtimes)}.")

Benchmarking TF model...
Average latency (seconds): 21.270257890879993.


### ONNX model

In [12]:
for _ in range(2):
    _ = sess.run(None, {"pixel_values": dummy_inputs_numpy})

# Timing
onnx_runtimes = timeit.repeat(
    lambda: sess.run(None, {"pixel_values": dummy_inputs_numpy}), number=1, repeat=25
)
print(f"Average latency (seconds): {np.mean(onnx_runtimes)}.")

Average latency (seconds): 8.338447264919996.
